# 하이퍼파라미터 수동으로 지정하기

- **우선적으로 시도**해야 할 하이퍼파라미터 세트가 있다
- 이미 시도한 적 있는 하이퍼파라미터 세트일 수도 있다

#### Optuna는 2가지를 제공함
1. `enqueue_trial()` : 우선 탐색하고 싶은 하이퍼파라미터가 있을 때 전달
2. `add_trial()` : 탐색이 완료된 하이퍼파라미터 세트와 그 결과를 전달

## 시나리오 1. 하이퍼파라미터를 Optuna로 평가하기
- `사용할 수 있는 하이퍼파라미터 세트가 있지만 평가하지 않았고, 더 나은 하이퍼파라미터 세트를 찾기 위해 Optuna를 사용한다`는 가정

In [1]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

- 목적함수 정의

In [5]:
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y = True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size = 0.25)
    dtrain = lgb.Dataset(train_x, label = train_y)
    dvalid = lgb.Dataset(valid_x, label = valid_y)
    
    
    param = {
        'objective' : 'binary',
        'metric' : 'auc',
        'verbosity' : -1,
        'boosting_type' : 'gbdt',
        'bagging_fraction' : min(trial.suggest_float('bagging_fraction', 0.4, 1.0 + 1e-12), 1),
        'bagging_freq' : trial.suggest_int('bagging_freq', 0, 7),
        'min_child_samples' : trial.suggest_int('min_child_samples', 5, 100),
    }
    
    # Pruning Callback 추가
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'auc')
    gbm = lgb.train(
        param, dtrain, valid_sets = [dvalid], verbose_eval = False, callbacks = [pruning_callback]
    )
    
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    
    return accuracy

In [6]:
study = optuna.create_study(direction = 'maximize',
                           pruner = optuna.pruners.MedianPruner())

[I 2022-12-12 15:41:14,044] A new study created in memory with name: no-name-47f001f2-270e-4a70-81bb-211312d35b91


A new study created in memory with name: no-name-47f001f2-270e-4a70-81bb-211312d35b91


In [7]:
# study에 euqne_trial을 지정하기만 하면 된다.
# 이들을 먼저 테스트해본 다음 다른 하이퍼파라미터 집합을 탐색한다
study.enqueue_trial(
    {
        'bagging_fraction' : 1.0,
        'bagging_freq' : 0,
        'min_child_samples' : 20
    })

study.enqueue_trial({
    'bagging_fraction' : 0.75,
    'bagging_freq' : 5,
    'min_child_samples' : 20
})

import logging
import sys

optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
study.optimize(objective, n_trials = 100, timeout = 600)

C:\Users\dowra\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-12-12 15:41:14,786] Trial 0 finished with value: 0.9790209790209791 and parameters: {'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}. Best is trial 0 with value: 0.9790209790209791.


Trial 0 finished with value: 0.9790209790209791 and parameters: {'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}. Best is trial 0 with value: 0.9790209790209791.
Trial 0 finished with value: 0.9790209790209791 and parameters: {'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}. Best is trial 0 with value: 0.9790209790209791.


C:\Users\dowra\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-12-12 15:41:14,906] Trial 1 finished with value: 0.958041958041958 and parameters: {'bagging_fraction': 0.75, 'bagging_freq': 5, 'min_child_samples': 20}. Best is trial 0 with value: 0.9790209790209791.


Trial 1 finished with value: 0.958041958041958 and parameters: {'bagging_fraction': 0.75, 'bagging_freq': 5, 'min_child_samples': 20}. Best is trial 0 with value: 0.9790209790209791.
Trial 1 finished with value: 0.958041958041958 and parameters: {'bagging_fraction': 0.75, 'bagging_freq': 5, 'min_child_samples': 20}. Best is trial 0 with value: 0.9790209790209791.


C:\Users\dowra\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-12-12 15:41:14,988] Trial 2 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.5699404608279761, 'bagging_freq': 0, 'min_child_samples': 52}. Best is trial 0 with value: 0.9790209790209791.


...

Trial 97 pruned. Trial was pruned at iteration 0.
Trial 97 pruned. Trial was pruned at iteration 0.


[I 2022-12-12 15:41:20,230] Trial 98 pruned. Trial was pruned at iteration 16.


Trial 98 pruned. Trial was pruned at iteration 16.
Trial 98 pruned. Trial was pruned at iteration 16.


[I 2022-12-12 15:41:20,273] Trial 99 pruned. Trial was pruned at iteration 0.


Trial 99 pruned. Trial was pruned at iteration 0.
Trial 99 pruned. Trial was pruned at iteration 0.


# 시나리오 2 : 이미 평가된 하이퍼파라미터 활용

- 가정 : 이미 평가된 하이퍼파라미터 세트가 있으며, 결과가 바람직하지 않아 Optuna 사용을 고려한다
- 이 때 `optuna.study.Study.add_trial()`을 사용할 수 있음

In [9]:
study = optuna.create_study(direction = 'maximize', pruner = optuna.pruners.MedianPruner())

study.add_trial(
    optuna.trial.create_trial(
        params = {
            'bagging_fraction' : 1.0,
            'bagging_freq' : 0,
            'min_child_samples' : 20,
        },
        distributions = {
            'bagging_fraction' : optuna.distributions.FloatDistribution(0.4, 1.0 + 1e-12),
            'bagging_freq' : optuna.distributions.IntDistribution(0, 7),
            'min_child_samples' : optuna.distributions.IntDistribution(5, 100),
        },
        value = 0.94
    )
)
study.add_trial(
    optuna.trial.create_trial(
        params={
            "bagging_fraction": 0.75,
            "bagging_freq": 5,
            "min_child_samples": 20,
        },
        distributions={
            "bagging_fraction": optuna.distributions.FloatDistribution(0.4, 1.0 + 1e-12),
            "bagging_freq": optuna.distributions.IntDistribution(0, 7),
            "min_child_samples": optuna.distributions.IntDistribution(5, 100),
        },
        value=0.95,
    )
)
study.optimize(objective, n_trials=100, timeout=600)

[I 2022-12-12 15:46:18,553] A new study created in memory with name: no-name-46381711-b3f2-4d50-96e6-91b2c53edc18


A new study created in memory with name: no-name-46381711-b3f2-4d50-96e6-91b2c53edc18
A new study created in memory with name: no-name-46381711-b3f2-4d50-96e6-91b2c53edc18


[I 2022-12-12 15:46:18,674] Trial 2 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.41041432121187066, 'bagging_freq': 5, 'min_child_samples': 77}. Best is trial 2 with value: 0.965034965034965.


Trial 2 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.41041432121187066, 'bagging_freq': 5, 'min_child_samples': 77}. Best is trial 2 with value: 0.965034965034965.
Trial 2 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.41041432121187066, 'bagging_freq': 5, 'min_child_samples': 77}. Best is trial 2 with value: 0.965034965034965.


C:\Users\dowra\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-12-12 15:46:18,755] Trial 3 finished with value: 0.958041958041958 and parameters: {'bagging_fraction': 0.42852161909238506, 'bagging_freq': 4, 'min_child_samples': 64}. Best is trial 2 with value: 0.965034965034965.


...

Trial 99 pruned. Trial was pruned at iteration 1.
Trial 99 pruned. Trial was pruned at iteration 1.


[I 2022-12-12 15:46:24,787] Trial 100 pruned. Trial was pruned at iteration 0.


Trial 100 pruned. Trial was pruned at iteration 0.
Trial 100 pruned. Trial was pruned at iteration 0.


[I 2022-12-12 15:46:24,943] Trial 101 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.7800947828105104, 'bagging_freq': 4, 'min_child_samples': 25}. Best is trial 20 with value: 0.993006993006993.


Trial 101 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.7800947828105104, 'bagging_freq': 4, 'min_child_samples': 25}. Best is trial 20 with value: 0.993006993006993.
Trial 101 finished with value: 0.965034965034965 and parameters: {'bagging_fraction': 0.7800947828105104, 'bagging_freq': 4, 'min_child_samples': 25}. Best is trial 20 with value: 0.993006993006993.
